<a href="https://colab.research.google.com/github/syrinegharbi/fast-bert-implementation-results/blob/master/fast_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install git+https://github.com/kaushaltrivedi/fast-bert.git



1- Create a DataBunch object
The databunch object takes training, validation and test csv files and converts the data into internal representation for BERT or XLNet. The object also instantiates the correct data-loaders based on device profile and batch_size and max_sequence_length.


In [0]:

from fast_bert.data_cls import BertDataBunch
from pathlib import Path
import torch
from fast_bert.learner_cls import BertLearner
from fast_bert.metrics import accuracy
import logging

torch.cuda.empty_cache()

DATA_PATH = Path('./')
LABEL_PATH = Path('./')
OUTPUT_PATH = Path('./')


databunch = BertDataBunch(DATA_PATH, LABEL_PATH,
                          tokenizer='bert-base-uncased',
                          train_file='trn.csv',
                          val_file='valid.csv',
                          label_file='lbl.csv',
                          text_col='Message',
                          label_col=["Corrective", "Perfective", "Adaptive"],
                          batch_size_per_gpu=4,
                          max_seq_length=512,
                          multi_gpu=True,
                          multi_label=True,
                          model_type='bert')
num_labels = len(databunch.labels)
print(num_labels)
torch.cuda.empty_cache()

**2 - Create a Learner Object**

In [0]:
logger = logging.getLogger()
device_cuda = torch.device("cuda")
metrics = [{'name': 'accuracy', 'function': accuracy}]

learner = BertLearner.from_pretrained_model(
						databunch,
						pretrained_path='bert-base-uncased',
						metrics=metrics,
						device=device_cuda,
						logger=logger,
						output_dir=OUTPUT_PATH,
						finetuned_wgts_path=None,
						warmup_steps=20,
						multi_gpu=False,
						is_fp16=True,
						multi_label=True,
						logging_steps=30)
torch.cuda.empty_cache()

In [0]:
!git clone https://github.com/NVIDIA/apex

In [0]:
!cd apex && pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

**Train the model**

In [0]:
torch.cuda.empty_cache()

In [0]:
learner.fit(epochs=6,
			lr=6e-5,
			validate=True, 	# Evaluate the model after each epoch
			schedule_type="warmup_cosine")
torch.cuda.empty_cache()

**Save the model**

In [0]:
learner.save_model()

**Prepare text to predict**

In [0]:
texts = ['add new interface and fix activity bug',
		 'push new version']
predictions = learner.predict_batch(texts)
print(predictions)